In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from crossValidation import GridSearch, CrossValidation, string_to_numerical_categories
from softmax import SoftmaxRegression, GradientDescent
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Iris Dataset
We find the best hyper-parameters by trying various combinations of batch size,
learning rate, and momentum, as seen below. We limit the number of options to speed up
the search.

In [2]:
# set to True for hypothesis test (discussed in the report)
normalize = False

In [3]:
iris = fetch_openml(name="iris")
iris["target"] = string_to_numerical_categories(iris["target"])

if normalize:
    scaler = StandardScaler()
    scaler.fit(iris['data'])
    iris['data'] = scaler.transform(iris['data'])

batch_sizes = [1, 5, 10, 20, 30]
learning_rates = [.0001, .001, .01, .1]
momentums = [.25, .5, .75, .95, .98, .99]

c:\users\julian\.virtualenvs\softmax_regression\lib\site-packages\sklearn\datasets\_openml.py:373: UserWarning: Multiple active versions of the dataset matching the name iris exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


Perform grid search to find the best parameters from the above sets. This may take a
couple of minutes to run.

In [4]:
search = GridSearch(
    5,
    iris,
    batch_sizes,
    learning_rates,
    momentums,
    max_iters=1e3
)
_, (_, best_val_params) = search.accuracy_plot()
batch_size, learning_rate, momentum = best_val_params
print("Best batch size: ", batch_size)
print("Best learning rate: ", learning_rate)
print("Best momentum: ", momentum)

Best batch size:  5
Best learning rate:  0.1
Best momentum:  0.25


Now that we have the best parameters, we compare softmax regression to K-Nearest Neighbors
using cross validation.

To find the best K, we perform a grid search for K = {1, ..., 20}

In [5]:

CV = CrossValidation(5, iris)

#find best knn model
k_values = np.zeros(50)
for i in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=i)
    _, k_values[i-1], _ = CV.cross_validate_acc(knn)
best_k = np.argmax(k_values) + 1

The best K for KNN is found to be:

In [6]:
print(best_k)

5


In [7]:


# compare models
gd = GradientDescent(learning_rate=learning_rate, batch_size=batch_size, momentum=momentum, max_iters=1e3)
softmax = SoftmaxRegression()
_, softmax_acc, _ = CV.cross_validate_acc(softmax, gd)

The 5-fold cross-validation accuracy of softmax regression is:

In [8]:
print(softmax_acc)

0.9866666666666667


And the 5-fold cross-validation accuracy of KNN is:

In [9]:
print(k_values[best_k])


0.9533333333333334
